In [1]:
import glob
import os
import pandas as pd
import numpy as np
import re
import shutil

# 1. Generate conformers for logp experimental

In [2]:
logpExp = pd.read_csv('/beegfs/dz1061/datasets/logp/logp_all_processed.csv')

In [5]:
logpExp.shape

(11281, 3)

1) generate conformers for each molecule that are stored in one sdf file. 

2) separate out each conformer in the sdf file in the form of single sdf file.
- To be noted, we separate 2000 files a time since one molecule's sdf file is able to produce large number of files. 

In [2]:
from genGaus import *

In [3]:
num_suc, failMol = file_seperate('//beegfs/dz1061/datasets/solvation_exp/makeups_confors_sdf/', '/beegfs/dz1061/datasets/solvation_exp/makeups_confors_sep_sdf/')

In [4]:
num_suc

1

3) generate com files. 

In [6]:
df_gas = pd.read_csv('/scratch/dz1061/gcn/logpExp/logp_part1_gas_opt.csv')

In [6]:
df_gas.head()

,enerGas,molID,confID
0,-5058.120511,0,3
1,-5903.294881,1,1
2,-3428.069975,2,79
3,-3307.769873,3,1
4,-3239.216382,4,10


In [7]:
for molid, confid in zip(df_gas['molID'], df_gas['confID']):
    #print(molid, confid)
    name = str(molid) + '_confors_' + str(confid) + '_opt_gas'
    
    file = '/scratch/dz1061/gcn/logpExp/gas_opt_out/' + str(molid) + \
    '_confors_' + str(confid) + '_opt_gas_opt.log'
    cmd = '/share/apps/openbabel/3.0.0/intel/bin/obabel -ig16 ' + file + ' -ocom -O ' + '/scratch/dz1061/gcn/logpExp/gas_opt_com/' + name + '_opt.com'
    os.system(cmd)
    #break

In [4]:
gas_com = glob.glob('/beegfs/dz1061/datasets/Frag20/Frag20_nolarger10_data/pubchem_gas_opt_com/*com')

In [24]:
files_unfinished = []

In [25]:
for file in glob.iglob('/beegfs/dz1061/datasets/Frag20/Frag20_nolarger10_data/pubchem/*log'):
    #print(file.split('/')[-1].split('.')[0])
    name = file.split('/')[-1].split('.')[0]
    #print(name)
    file_name = '/beegfs/dz1061/datasets/Frag20/Frag20_nolarger10_data/pubchem_gas_opt_com/' + str(name) +'_gas_opt.com'
    if file_name not in gas_com:
        files_unfinished.append(file)
    #print(name)
    #print(file)
    #cmd = '/share/apps/openbabel/3.0.0/intel/bin/obabel -ig16 ' + file + ' -ocom -O ' + '/beegfs/dz1061/datasets/solvation_exp/makeups_gas_opt_com/' + name + '_opt.com'
    #os.system(cmd)
    
    #break

In [26]:
len(files_unfinished)

2266

In [28]:
for file in files_unfinished:
    name = file.split('/')[-1].split('.')[0]
    cmd = '/share/apps/openbabel/3.0.0/intel/bin/obabel -ig09 ' + file + ' -ocom -O ' + '/beegfs/dz1061/datasets/Frag20/Frag20_nolarger10_data/pubchem_gas_opt_com/' + name + '_gas_opt.com'
    os.system(cmd)

Need to check com file to gaussian file in the next steps

In [14]:
def gaussian_gen(datadir, out_dir, phase='octanol'):
    """ 
    Add header for guassian optimization  
    header_file: the file with Gaussian calculation header information
    """
    for file in glob.iglob(os.path.join(datadir, '*.com')):
        #print(file)
        name = file.split('/')[-1].split('.')[0]
        
        a = []
        temp = []
        with open(file) as f:
            for line in f.readlines():
                a.append(line)
        if a:      
            check_point='%Chk=' + '"' + name + '_' + phase + '"'
            cpu = '%NProcShared=2'
            mem = '%Mem=1GB'
            #method = '#  B3LYP/6-31G* scrf=(smd, solvent=water)'
            method = '#  B3LYP/6-31G* scrf=(smd, solvent=n-octanol)'
            #method = '# opt freq B3LYP/6-31G*'
            space = ' '
            note = name+' octanol phase'

            with open(out_dir + name + '_' + phase + '.gjf', 'w') as file:
                temp_claim = [check_point,cpu,mem,method,space,note,space]
                for item in temp_claim:
                    file.write('%s\n' % item)
                for line in a[5:]:
                    file.write(line)

In [12]:
gaussian_gen('/scratch/dz1061/gcn/logpExp/gas_opt_com', '/scratch/dz1061/gcn/logpExp/water_smd_gjf//')

In [15]:
gaussian_gen('/scratch/dz1061/gcn/logpExp/gas_opt_com', '/scratch/dz1061/gcn/logpExp/octanol_smd_gjf/')

In [16]:
def gen_in(path_in, path_out, path_result, phase):
    
    if phase=='gas':
        fileList = glob.glob(path_in)
        print(len(fileList))
        num = int(len(fileList) / 20)
        for k in range(num+1):
            print((k+1)*20)
            with open(path_out + str(k) + '_20.in','w+') as f:
            #for file in os.listdir(path_in): 
                    #if file.endswith('.gjf'):
                        #with open(path_out + file[:-4] + '.in','w+') as f:
                                line1 = '#!/bin/bash'
                                line2 = '#'
                                line3 = '#SBATCH --job-name=' + str(k)
                                line4 = '#SBATCH --nodes=1'
                                line5 = '#SBATCH --cpus-per-task=2'
                                line6 = '#SBATCH --time=100:00:00'
                                line7 = '#SBATCH --mem=1GB'
                                linespace = ' '
                                line8 = 'module purge'
                                line9 = 'module load gaussian/intel/g16a03'
                                line10 = 'cd /scratch/dz1061/gcn/logpExp/gas_opt_gjf'
                                lines = [line1,line2,line3,line4,line5,line6,line7,linespace,linespace,line8,line9,linespace,line10]
                                for line in lines:
                                    f.write("%s\n" % line)
                                for file in fileList[k*20:(k+1)*20]:
                                    name = file.split('/')[-1].split('.')[0]
                                    #print(name)
                                    line11='srun run-gaussian ' + file  + ' > ' + path_result + name + '_opt.log' + ' 2>&1' ' && mv ' + file + ' ' + '/scratch/dz1061/gcn/logpExp/gas_opt_gjf_finished/;'
                                    f.write("%s\n" % line11)
                                
    if phase=='water':
        fileList = glob.glob(path_in)
        print(len(fileList))
        num = int(len(fileList) / 20)
        for k in range(num+1):
            print((k+1)*20)
            with open(path_out + str(k) + '_20.in','w+') as f:
            #for file in os.listdir(path_in): 
                    #if file.endswith('.gjf'):
                        #with open(path_out + file[:-4] + '.in','w+') as f:
                                line1 = '#!/bin/bash'
                                line2 = '#'
                                line3 = '#SBATCH --job-name=' + str(k)
                                line4 = '#SBATCH --nodes=1'
                                line5 = '#SBATCH --cpus-per-task=2'
                                line6 = '#SBATCH --time=30:00:00'
                                line7 = '#SBATCH --mem=1GB'
                                linespace = ' '
                                line8 = 'module purge'
                                line9 = 'module load gaussian/intel/g16a03'
                                line10 = 'cd /scratch/dz1061/gcn/logpExp/water_smd_gjf'
                                lines = [line1,line2,line3,line4,line5,line6,line7,linespace,linespace,line8,line9,linespace,line10]
                                for line in lines:
                                    f.write("%s\n" % line)
                                for file in fileList[k*20:(k+1)*20]:
                                    name = file.split('/')[-1].split('.')[0]
                                    #print(name)
                                    line11='srun run-gaussian ' + file  + ' > ' + path_result + name + '_smd.log' + ' 2>&1' ' && mv ' + file + ' ' + '/scratch/dz1061/gcn/logpExp/water_smd_gjf_finished;'
                                    f.write("%s\n" % line11)
            #break
                                
    if phase=='octanol':
        fileList = glob.glob(path_in)
        print(len(fileList))
        num = int(len(fileList) / 20)
        for k in range(num+1):
            print((k+1)*20)
            with open(path_out + str(k) + '_20.in','w+') as f:
            #for file in os.listdir(path_in): 
                    #if file.endswith('.gjf'):
                        #with open(path_out + file[:-4] + '.in','w+') as f:
                                line1 = '#!/bin/bash'
                                line2 = '#'
                                line3 = '#SBATCH --job-name=' + str(k)
                                line4 = '#SBATCH --nodes=1'
                                line5 = '#SBATCH --cpus-per-task=2'
                                line6 = '#SBATCH --time=30:00:00'
                                line7 = '#SBATCH --mem=1GB'
                                linespace = ' '
                                line8 = 'module purge'
                                line9 = 'module load gaussian/intel/g16a03'
                                line10 = 'cd /scratch/dz1061/gcn/logpExp/octanol_smd_gjf'
                                lines = [line1,line2,line3,line4,line5,line6,line7,linespace,linespace,line8,line9,linespace,line10]
                                for line in lines:
                                    f.write("%s\n" % line)
                                for file in fileList[k*20:(k+1)*20]:
                                    name = file.split('/')[-1].split('.')[0]
                                    #print(name)
                                    line11='srun run-gaussian ' + file  + ' > ' + path_result + name + '_smd.log' + ' 2>&1' ' && mv ' + file + ' ' + '/scratch/dz1061/gcn/logpExp/octanol_smd_gjf_finished;'
                                    f.write("%s\n" % line11)

In [29]:
'/scratch/dz1061/gcn/logpExp/water_smd_gjf' + '_'

'/scratch/dz1061/gcn/logpExp/water_smd_gjf_'

In [1]:
#gen_in('/scratch/dz1061/gcn/logpExp/water_smd_gjf/*gjf', '/scratch/dz1061/gcn/logpExp/water_smd_in/', '/scratch/dz1061/gcn/logpExp/water_smd_out/', phase='water')